<a href="https://colab.research.google.com/github/shtoshni92/fast-coref/blob/main/On_Generalization_in_Coreference_Resolution_(CRAC_2021).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This colab notebook performs inference with a model trained on LitBank described in our EMNLP 2020 paper [Learning to Ignore: Long Document Coreference with Bounded Memory Neural Networks](https://www.aclweb.org/anthology/2020.emnlp-main.685.pdf)

### Clone Github repo

In [1]:
%%capture
! git clone https://github.com/shtoshni92/fast-coref.git

### Install Relevant Libraries

In [9]:
%%capture
! pip install torch==1.10.0
! pip install transformers==4.11.3
! pip install spacy==3.0.5
! python -m spacy download en_core_web_sm
! pip install scipy==1.6.1
! pip install omegaconf==2.1.1

### Download Pretrained Models

The pretrained models are released [here](https://drive.google.com/drive/folders/1270pP1JIYLleLH7rkRyXyHV2p0C7rX_8?usp=sharing). In this example, we will be using the OntoNotes-only model (longdoc + speaker) which gets a CoNLL score of 80.8 on the test set and 80.5 on the validation set.

In [5]:
!gdown --id 1CQxUq2zvCHc1mJUEZ_Zy6WSJQqFz76Pw

Downloading...
From: https://drive.google.com/uc?id=1CQxUq2zvCHc1mJUEZ_Zy6WSJQqFz76Pw
To: /content/model.pth
100% 149M/149M [00:01<00:00, 95.0MB/s]


In [6]:
ls

fast-coref/  model.pth  sample_data/


### Inference on Sample Text

Now that we have the code and the pretrained model, time to test the model on some sample data. 

In [11]:
import sys
sys.path.append('fast-coref/src')

# This will also download the Longformer model finetuned for OntoNotes from Huggingface
from inference.model_inference import Inference
inference_model = Inference("./", encoder_name="shtoshni/longformer_coreference_ontonotes")

Downloading:   0%|          | 0.00/803 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/50.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/306 [00:00<?, ?B/s]

### Sample Doc

Here's an excerpt from [The War of the Worlds](https://en.wikipedia.org/wiki/The_War_of_the_Worlds). The LitBank annotations for the doc are visualized [here](https://ttic.uchicago.edu/~shtoshni/coref/litbank_html/36_the_war_of_the_worlds.html) -- the excerpt is just the prefix (The doc was part of the dev set for the LitBank model we are using).


In [12]:
doc = """
    BOOK ONE THE COMING OF THE MARTIANS CHAPTER ONE THE EVE OF THE WAR No one would have believed in the last years of the nineteenth century that this world was being watched keenly and closely by intelligences greater than man 's and yet as mortal as his own ; that as men busied themselves about their various concerns they were scrutinised and studied , perhaps almost as narrowly as a man with a microscope might scrutinise the transient creatures that swarm and multiply in a drop of water .
    With infinite complacency men went to and fro over this globe about their little affairs , serene in their assurance of their empire over matter .
    It is possible that the infusoria under the microscope do the same .
    No one gave a thought to the older worlds of space as sources of human danger , or thought of them only to dismiss the idea of life upon them as impossible or improbable .
    It is curious to recall some of the mental habits of those departed days .
    At most terrestrial men fancied there might be other men upon Mars , perhaps inferior to themselves and ready to welcome a missionary enterprise .
    Yet across the gulf of space , minds that are to our minds as ours are to those of the beasts that perish , intellects vast and cool and unsympathetic , regarded this earth with envious eyes , and slowly and surely drew their plans against us .
    And early in the twentieth century came the great disillusionment .
    The planet Mars , I scarcely need remind the reader , revolves about the sun at a mean distance of 140,000,000 miles , and the light and heat it receives from the sun is barely half of that received by this world .
    It must be , if the nebular hypothesis has any truth , older than our world ; and long before this earth ceased to be molten , life upon its surface must have begun its course .
    The fact that it is scarcely one seventh of the volume of the earth must have accelerated its cooling to the temperature at which life could begin .
    It has air and water and all that is necessary for the support of animated existence .
    Yet so vain is man , and so blinded by his vanity , that no writer , up to the very end of the nineteenth century , expressed any idea that intelligent life might have developed there far , or indeed at all , beyond its earthly level .
    Nor was it generally understood that since Mars is older than our earth , with scarcely a quarter of the superficial area and remoter from the sun , it necessarily follows that it is not only more distant from time 's beginning but nearer its end .
    The secular cooling that must someday overtake our planet has already gone far indeed with our neighbour .
    Its physical condition is still largely a mystery , but we know now that even in its equatorial region the midday temperature barely approaches that of our coldest winter .
    Its air is much more attenuated than ours , its oceans have shrunk until they cover but a third of its surface , and as its slow seasons change huge snowcaps gather and melt about either pole and periodically inundate its temperate zones .
    That last stage of exhaustion , which to us is still incredibly remote , has become a present-day problem for the inhabitants of Mars .
    """

In [13]:
output = inference_model.perform_coreference(doc)

In [14]:
for cluster in output["clusters"]:
  print(cluster)

[((8, 24), 'THE COMING OF THE MARTIANS CHAPTER ONE THE EVE OF THE WAR')]
[((8, 16), 'THE COMING OF THE MARTIANS')]
[((12, 16), 'THE MARTIANS')]
[((20, 24), 'THE EVE OF THE WAR')]
[((35, 37), 'the nineteenth century')]
[((39, 40), 'this world'), ((124, 125), 'this globe'), ((299, 300), 'this earth'), ((384, 385), 'this world'), ((408, 409), 'our world'), ((414, 415), 'this earth'), ((423, 423), 'its'), ((449, 450), 'the earth'), ((560, 561), 'our earth'), ((611, 612), 'our planet')]
[((53, 55), "man 's"), ((61, 61), 'his')]
[((61, 62), 'his own')]
[((66, 66), 'men'), ((69, 69), 'themselves'), ((71, 71), 'their'), ((74, 74), 'they')]
[((118, 118), 'men'), ((127, 127), 'their'), ((134, 134), 'their'), ((137, 137), 'their')]
[((175, 179), 'the older worlds of space'), ((189, 189), 'them'), ((198, 198), 'them')]
[((179, 179), 'space')]
[((221, 223), 'those departed days')]
[((233, 234), 'terrestrial men'), ((248, 248), 'themselves')]
[((243, 243), 'Mars'), ((340, 342), 'The planet Mars'), (

#### Remove Singletons


In [ ]:
for cluster in output["clusters"]:
  if len(cluster) > 1:
    print(cluster)

[((397, 397), 'man'), ((403, 403), 'his')]
[((547, 548), 'its oceans'), ((552, 552), 'they')]
[((28, 29), 'this world'), ((100, 101), 'this globe'), ((237, 238), 'this earth'), ((308, 309), 'this world'), ((325, 326), 'our world'), ((331, 332), 'this earth'), ((360, 361), 'the earth'), ((453, 454), 'our earth')]
[((40, 40), 'man'), ((48, 48), 'his')]
[((53, 53), 'men'), ((55, 55), 'themselves'), ((57, 57), 'their'), ((60, 60), 'they'), ((94, 94), 'men'), ((103, 103), 'their'), ((109, 109), 'their'), ((112, 112), 'their'), ((248, 248), 'their')]
[((136, 140), 'the older worlds of space'), ((150, 150), 'them'), ((159, 159), 'them')]
[((182, 183), 'terrestrial men'), ((196, 196), 'themselves')]
[((191, 191), 'Mars'), ((264, 266), 'The planet Mars'), ((296, 296), 'it'), ((311, 311), 'It'), ((340, 340), 'its'), ((351, 351), 'it'), ((365, 365), 'its'), ((376, 376), 'It'), ((429, 429), 'there'), ((438, 438), 'its'), ((449, 449), 'Mars'), ((474, 474), 'it'), ((505, 506), 'our neighbour'), ((50

In [18]:
print(output['tokenized_doc']['orig_tokens'])

['\n    ', 'BOOK', 'ONE', 'THE', 'COMING', 'OF', 'THE', 'MARTIANS', 'CHAPTER', 'ONE', 'THE', 'EVE', 'OF', 'THE', 'WAR', 'No', 'one', 'would', 'have', 'believed', 'in', 'the', 'last', 'years', 'of', 'the', 'nineteenth', 'century', 'that', 'this', 'world', 'was', 'being', 'watched', 'keenly', 'and', 'closely', 'by', 'intelligences', 'greater', 'than', 'man', "'s", 'and', 'yet', 'as', 'mortal', 'as', 'his', 'own', ';', 'that', 'as', 'men', 'busied', 'themselves', 'about', 'their', 'various', 'concerns', 'they', 'were', 'scrutinised', 'and', 'studied', ',', 'perhaps', 'almost', 'as', 'narrowly', 'as', 'a', 'man', 'with', 'a', 'microscope', 'might', 'scrutinise', 'the', 'transient', 'creatures', 'that', 'swarm', 'and', 'multiply', 'in', 'a', 'drop', 'of', 'water', '.', '\n    ', 'With', 'infinite', 'complacency', 'men', 'went', 'to', 'and', 'fro', 'over', 'this', 'globe', 'about', 'their', 'little', 'affairs', ',', 'serene', 'in', 'their', 'assurance', 'of', 'their', 'empire', 'over', 'matt